Dataprep 是一个开源 Python 库，可以帮助我们自动实现一些EDA的工作。
这是他的官网，https://dataprep.ai/

In [ ]:
!pip install dataprep

In [ ]:
import numpy as np 
import pandas as pd 
from dataprep.eda import plot, plot_correlation, plot_missing

In [ ]:
train_df=pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
test_df=pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")

# 数据分布

In [ ]:
#train
plot(train_df)

In [ ]:
#test
plot(test_df)

在训练级中有一部分数据是正态分布的（f72）,还有一些数据有多个峰值（f94）,最主要的一点是大部分数据都是偏态分布，所以如果我们需要使用线性模型的话就需要注意这些数据分布的问题。

在训练集中这些列有着相似的分布
f4 - f22 ，f4 - f75 ，f5 - f99 ，f13 - f99 ，f22 - f75 ，f23 - f29 ，f23 - f46 ，f23 - f88 ，f25 - f36 ，f29 - f88 ，f46 - f88，f54 - f59，f56 - f91，f68 - f95，f83 - f87

再看一下测试集：
f3 - f32，f4 - f22，f5 - f13，f5 - f99，f6 - f35，f9 - f30，f13 - f99，f22 - f75，f23 - f29，f23 - f46，f23 - f88，f26 - f34，f46 - f88，f54 - f59，f56 - f91，f61 - f72，f63 - f66，f68 - f95，f83 - f87

找出在数据集中分布相似的列：
* f4 - f22
* f5 - f13 - f99 
* f22 - f75
* f23 - f29
* f23 - f46  - f88 
* f56 - f91
* f68 - f95
* f83 - f87

这对于这些列，后续可以进行更深入的研究

对于目标值（loss），还有一点需要强调指出：有24%的值是0，后面目标部分会有一些想法

# 相关性

In [ ]:
 plot_correlation(train_df)

In [ ]:
 plot_correlation(test_df)

相关系数的绝对值越大，相关性越强，基本上都是弱相关

# 缺失值

In [ ]:
plot_missing(train_df)

In [ ]:
plot_missing(test_df)

没有缺失值，不需要处理了

# 数据类型

In [ ]:
train_df.dtypes

In [ ]:
test_df.dtypes

In [ ]:
for i in range(len(train_df.dtypes)):
    t=train_df.dtypes[i]
    if (t!=np.float64):
        print(train_df.columns[i],t)

以上几列都是int，我们看看数值：
以下代码来自：https://www.kaggle.com/josetorrado/rookie-smarts-a-beginner-s-perspective

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
integer_fields = ['f1', 'f16', 'f27', 'f55', 'f60', 'f86']
train_unique = pd.DataFrame(train_df[integer_fields].nunique())
train_unique = train_unique.reset_index(drop=False)
train_unique.columns = ['Features', 'Count']
test_unique = pd.DataFrame(test_df[integer_fields].nunique())
test_unique = test_unique.reset_index(drop=False)
test_unique.columns = ['Features', 'Count']
sns.set_style("dark")
plot = sns.barplot(x= train_unique.Features, y=train_unique.Count)

plt.title('Count of Unique Values - Train')
plt.xlabel('Features')
plt.ylabel('Count')
plt.xticks(rotation=30, horizontalalignment="center")
plt.figure(figsize=(20,20))

x = train_unique['Features'].values.tolist()
y = train_unique['Count'].values.tolist()

# Annotations
for bar in plot.patches:
    plot.annotate(format(bar.get_height(), '.0f'), 
                  (bar.get_x() + bar.get_width()/2, bar.get_height()), 
                  ha='center', va='center', size='10', xytext=(0,8), 
                  textcoords='offset points')
plt.show()

虽然是整数，但是数值还是很多的，无法使用embbedding，因为太多了，使用onehot也不太现实

# 目标
loss是我们的目标，这意味着我们将预测一个数字，第一眼看到这是一个回归问题，因为我们要输出的是一个数字，并且评价的指标是RMSE，也是明显的回归指标。但是，实际上呢，这个loss值是int的整型，那不就是分类的label吗，我们再仔细研究下。

In [ ]:
cat=train_df["loss"].unique()
len(cat),cat

一共43个唯一值并且都是整型

In [ ]:
train_df["loss"].value_counts()

这明显是一个不平衡分类的问题，如果要想提高分数，在目标的选择上下手绝对会有所帮助

# 数据缩放
如果不使用基于树的模型，还需要下一步的操作：数据缩放。强调一点，test和train要一同处理。

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
features = [f'f{i}' for i in range(100)]
train_df[features] = ss.fit_transform(train_df[features])
test_df[features] = ss.transform(test_df[features])

In [ ]:
train_df